In [1]:
:set -XOverloadedStrings

import Control.Monad.Trans.Reader (runReaderT)
import Control.Monad.IO.Class (liftIO)
import Data.Maybe (fromJust)
import qualified Data.Set as S

import Duffer
import Duffer.Loose.Objects

duffer = flip runReaderT ".git"

Let's start with a raw representation of the most recent commit:

In [2]:
:!git show --format=raw -s

commit e21e9e098c9c70724d9fb9f075723f81e59f178d
tree e5eceecb4f78cdd7ddb97de26515ba753f125d5a
parent db7d3b03ea73d1f78b8e5b463a69024d48c3bb14
author Vaibhav Sagar <vaibhavsagar@gmail.com> 1475072814 -0400
committer Vaibhav Sagar <vaibhavsagar@gmail.com> 1475072814 -0400

    Add latest demo and presentation.

I'm currently on the `master` branch, so another way to get to this object is as follows:

In [3]:
duffer (resolveRef "refs/heads/master")

tree e5eceecb4f78cdd7ddb97de26515ba753f125d5a
parent db7d3b03ea73d1f78b8e5b463a69024d48c3bb14
author Vaibhav Sagar <vaibhavsagar@gmail.com> 1475072814 -0400
committer Vaibhav Sagar <vaibhavsagar@gmail.com> 1475072814 -0400

Add latest demo and presentation.

We can view the pretty-printed contents of a `git` object with `cat-file -p`. Each commit has a `tree` associated with it which represents a directory, in this case the root project folder.

In [4]:
:!git cat-file -p master^{tree}

040000 tree b8c0fd17ebd7bad61687fe06449b0a484b207efb	.ci
100644 blob 024ac699940a0c2da0a20f899b324df16f8fdeca	.gitignore
100644 blob fbc4e9eb55e7511e5dc1dc34a511d03f40c7cbd2	.travis.yml
100644 blob d5c01fc4f0a44b01144a7adb8d7eb9698870dbf3	Index.ipynb
100644 blob 40047c95db5ba581586d5ced7b041b64a9635512	LICENSE
100644 blob 96d207607fb9cc235de412f21594feb5710665a3	README.md
100644 blob 9a994af677b0dfd41b4e3b76b3e7e604003d64e1	Setup.hs
100644 blob 54e5297c419c544fa5003cfbeba70a4707319f52	demo.ipynb
100644 blob c227ff4dfbff1db536cad6764419cadcb8b16e5d	duffer.cabal
040000 tree 99b7e6ba8585ea0ee612b3bdc48d14d387e2418c	presentation
040000 tree ba3fbeb785137064219fc944a8a656c9b3b8efcd	src
100644 blob f455b1ce34e0111681afd47e50280d8b4841d810	stack.yaml
040000 tree cb611148ac47d79ace84e4604a1aef387bd40514	test

Again, we can obtain almost identical (modulo formatting) output with `duffer`:

In [5]:
duffer $ do
    master    <- resolveRef "refs/heads/master"
    let tree  =  treeRef master
    fromJust <$> readObject tree

040000	tree	b8c0fd17ebd7bad61687fe06449b0a484b207efb	.ci
100644	blob	024ac699940a0c2da0a20f899b324df16f8fdeca	.gitignore
100644	blob	fbc4e9eb55e7511e5dc1dc34a511d03f40c7cbd2	.travis.yml
100644	blob	d5c01fc4f0a44b01144a7adb8d7eb9698870dbf3	Index.ipynb
100644	blob	40047c95db5ba581586d5ced7b041b64a9635512	LICENSE
100644	blob	96d207607fb9cc235de412f21594feb5710665a3	README.md
100644	blob	9a994af677b0dfd41b4e3b76b3e7e604003d64e1	Setup.hs
100644	blob	54e5297c419c544fa5003cfbeba70a4707319f52	demo.ipynb
100644	blob	c227ff4dfbff1db536cad6764419cadcb8b16e5d	duffer.cabal
040000	tree	99b7e6ba8585ea0ee612b3bdc48d14d387e2418c	presentation
040000	tree	ba3fbeb785137064219fc944a8a656c9b3b8efcd	src
100644	blob	f455b1ce34e0111681afd47e50280d8b4841d810	stack.yaml
040000	tree	cb611148ac47d79ace84e4604a1aef387bd40514	test

`git` implements a giant hashtable on the filesystem using SHA1 as the hashing function. It stores all the past files and directory listings as `zlib`-compressed text files (with a header denoting object type and length) under `.git/objects` as follows:

1. Compute a SHA1 hash of the content.
2. `zlib`-compress the content.
3. Take the first 2 characters of the hash. This is the subdirectory under `.git/objects` where the content will be stored.
4. The remaining 38 characters of the hash are the filename.

For example, a decompressed commit looks like:

In [6]:
:!cat .git/objects/4b/d9b179bb166b85e3e889f9f263f1b5a26f3e34 | zlib-flate -uncompress

commit 287 tree 0b36647819f93c5523b5967d19cb131d88ab1be4
parent 2577894fb379a7cbe2e3bfd3ba325f4e451bbb5f
parent d72ae27a9ae58d49235ff9761cfae816b004d9b1
author Vaibhav Sagar <vaibhavsagar@gmail.com> 1473766850 -0400
committer Vaibhav Sagar <vaibhavsagar@gmail.com> 1473766850 -0400

Add porcelain.

In [7]:
fmap fromJust $ duffer $ readObject "4bd9b179bb166b85e3e889f9f263f1b5a26f3e34"

tree 0b36647819f93c5523b5967d19cb131d88ab1be4
parent 2577894fb379a7cbe2e3bfd3ba325f4e451bbb5f
parent d72ae27a9ae58d49235ff9761cfae816b004d9b1
author Vaibhav Sagar <vaibhavsagar@gmail.com> 1473766850 -0400
committer Vaibhav Sagar <vaibhavsagar@gmail.com> 1473766850 -0400

Add porcelain.

In [8]:
:!git branch

  gh-pages
* master

In [9]:
duffer $ do
    current       <- resolveRef "refs/heads/master"
    Just parent   <- current Duffer.^^ 1
    Just ancestor <- parent ~~ 1
    return ancestor

tree b6a522a22b9606aa5e63304b8e9bb24d4c2e52d2
parent bb6c2f24de25d142a674cb73253cca7233ff4f33
author Vaibhav Sagar <vaibhavsagar@gmail.com> 1474940870 -0400
committer Vaibhav Sagar <vaibhavsagar@gmail.com> 1474940870 -0400

First of many changes to presentation.

As mentioned previously, the hash of a `git` object uniquely identifies it in the giant hashtable that is `git`

In [10]:
tree <- fmap fromJust $ duffer $ readObject "a28aded05daa52ff5d0c77cd6186b1ce0faf7c8c"
hash tree

"a28aded05daa52ff5d0c77cd6186b1ce0faf7c8c"

`git` refers to files as `blob`s.

In [11]:
fmap fromJust $ duffer $ readObject "b75f4c9dbe3b61cacba052f23461834468832e41"

Copyright Vaibhav Sagar (c) 2015

All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

    * Redistributions of source code must retain the above copyright
      notice, this list of conditions and the following disclaimer.

    * Redistributions in binary form must reproduce the above
      copyright notice, this list of conditions and the following
      disclaimer in the documentation and/or other materials provided
      with the distribution.

    * Neither the name of Vaibhav Sagar nor the names of other
      contributors may be used to endorse or promote products derived
      from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
"AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
A PARTICULAR PURPOSE ARE DIS

The last type of `git` object is a `tag`, which gives a name to another `git` object.

In [12]:
fmap fromJust $ duffer $ readObject "d4b1e0343313ab60688cf0ddfa8ae5d8fe60ec23"

object 25354a5cfebca0261cdaa87ebef3a6b9dcb9c13a
type commit
tag test
tagger Vaibhav Sagar <vaibhavsagar@gmail.com> 1459935215 +1000

Test tag.

`duffer` is pretty great at reading `git` repositories, but that's not all you can do with it. You can also add content to a `git` repository with it:

In [13]:
import Data.ByteString.UTF8 (fromString, toString)
blob = Blob (fromString "hello world")
duffer $ writeObject blob

"95d09f2b10159347eece71399a7e2e907ea3df4f"

In [14]:
:!git cat-file -p 95d09f2b10159347eece71399a7e2e907ea3df4f

hello world

In [15]:
:!git branch

  gh-pages
* master

In [16]:
currentCommit = resolveRef "refs/heads/master"
duffer $ currentCommit >>= \commit -> updateRef "refs/heads/new-branch" commit

"e21e9e098c9c70724d9fb9f075723f81e59f178d"

In [17]:
:!git branch

  gh-pages
* master
  new-branch

In [18]:
setMSBs :: [Int] -> [Int]
setMSBs ints = let
    ints'  = reverse ints
    ints'' = head ints' : map (`setBit` 7) ( tail ints')
in reverse ints''rootTree = duffer $ do
    ref <- treeRef <$> currentCommit
    fromJust <$> readObject ref
rootTree

Parse error (line 4, column 57): parse error (possibly incorrect indentation or mismatched brackets)

Parse error (line 5, column 1): parse error on input ‘in’

In [19]:
newFile = TreeEntry 0o100644 "new-file" "95d09f2b10159347eece71399a7e2e907ea3df4f"
duffer $ do
    treeEntries <- liftIO $ entries <$> rootTree
    let newEntries = S.insert newFile treeEntries
    newTree <- writeObject (Tree newEntries)
    let me = PersonTime "Vaibhav Sagar" "vaibhavsagar@gmail.com" "1461156164" "+1000" 
    let newCommit = Commit newTree ["d76238fed6c656183a4d4dcf287217a061043869"] me me "New commit."
    newHead <- writeObject newCommit
    updateRef "refs/heads/new-branch" newCommit

Not in scope: ‘rootTree’

In [20]:
newTree = duffer $ do
    ref <- treeRef <$> resolveRef "refs/heads/new-branch"
    fromJust <$> readObject ref
newTree

040000	tree	b8c0fd17ebd7bad61687fe06449b0a484b207efb	.ci
100644	blob	024ac699940a0c2da0a20f899b324df16f8fdeca	.gitignore
100644	blob	fbc4e9eb55e7511e5dc1dc34a511d03f40c7cbd2	.travis.yml
100644	blob	d5c01fc4f0a44b01144a7adb8d7eb9698870dbf3	Index.ipynb
100644	blob	40047c95db5ba581586d5ced7b041b64a9635512	LICENSE
100644	blob	96d207607fb9cc235de412f21594feb5710665a3	README.md
100644	blob	9a994af677b0dfd41b4e3b76b3e7e604003d64e1	Setup.hs
100644	blob	54e5297c419c544fa5003cfbeba70a4707319f52	demo.ipynb
100644	blob	c227ff4dfbff1db536cad6764419cadcb8b16e5d	duffer.cabal
040000	tree	99b7e6ba8585ea0ee612b3bdc48d14d387e2418c	presentation
040000	tree	ba3fbeb785137064219fc944a8a656c9b3b8efcd	src
100644	blob	f455b1ce34e0111681afd47e50280d8b4841d810	stack.yaml
040000	tree	cb611148ac47d79ace84e4604a1aef387bd40514	test